The reviews seem to actually give the evaluation at the beginning and towards the end. The rest is mostly plot descriptions. Here I take the 20% from the start and towards the end and
insert that into the model, instead of the full review. Thus we significantly reduce the prompt size - by e.g. 60% but keep the evaluatory context.

-------------------------
-------------------------

Here are some examples:

ORIGINAL:

He is very good in this role as a disaffected and bored husband, a decorated air force officer, who becomes bored with his predictable suburban life and decides to make a radical change. One evening he meets Karen Sillas, an attractive woman younger than his current wife. A relationship develops, and Cole decides to take it to the next level. Not only does he have an affair, he fakes his death to escape his family obligations. Lies become increasingly more of a pattern as he begins to rob banks to keep his new wife (Sillas) happy, and to project the image that he is a success. A fictional delusion, apparently. This was also based on a true story which makes it all the more intriguing. Cole is believable and excellent in these roles. Recommended. 8/10.

CROPPED:

He is very good in this role as a disaffected and bored husband, a decorated air force officer, who becomes bored with his predictable suburban life and apparently. This was also based on a true story which makes it all the more intriguing. Cole is believable and excellent in these roles. Recommended. 8/10.

-------------------------

ORIGINAL:

My main comment on this movie is how Zwick was able to get credible actors to work on this movie? Impressive cast  even for the supporting characters, none of which helps this movie really. I have to admit though, Tom Hank's cameo almost made it worth it  what was that about Tom? Did you lose a bet? The best cameo of the movie was Joe Isuzu though - by far a classic! The premise is good. Basinger's character, struggling with existence as a Pink Lady, is making her way toward Vegas motel by motel pitching the glorious pyramid of cosmetic sales. This happens as Corbett's character is on his way to Vegas to deliver an Elvis suit to his soon to be ex-wife motivated by.what else.extortion. As they both make their way, they have numerous run-ins with Elvis impersonators who on their way to an Elvis impersonating convention in Vegas. Soon, the FBI gets involved and begins to track what they think is an Elvis impersonator serial killer. Unfortunately, premise doesn't mean the movie was good. When watching this movie, imagine you are back in the first grade  when story lines and continuity aren't really important. It is much more enjoyable to just watch Basinger look beautiful in her Pink Lady outfit rather than wondering why what she is doing doesn't really make sense. The movie tries hard, but ultimately falls way way way short. Ultimately, it is filled with ideas that could have theoretically been funny but in practice were not that funny. It isn't the worst, but you may find you yourself feel like leaving the building when watching this one Don't say I didn't warn you!

CROPPED:

My main comment on this movie is how Zwick was able to get credible actors to work on this movie? Impressive cast  even for the supporting characters, none of which helps this movie really. I have to admit though, Tom Hank's cameo almost made it worth it  what was that about Tom? Did you lose sense. The movie tries hard, but ultimately falls way way way short. Ultimately, it is filled with ideas that could have theoretically been funny but in practice were not that funny. It isn't the worst, but you may find you yourself feel like leaving the building when watching this one Don't say I didn't warn you!

---------------------------


In [19]:
import sys
sys.path.append("..")
from llama_cpp import Llama
import pandas as pd
from langdetect import detect
from mlflow_experiment import MlflowExperiment, EvaluationPipeline, InferencePipeline
from mlflow_experiment.evaluation.metrics import (accuracy_score, 
                                                  precision,
                                                  recall,
                                                  false_negative_rate,
                                                  false_positive_rate,
                                                  median_response_token_count,
                                                  median_query_token_count, 
                                                  median_processing_time, 
                                                  median_tokens_per_second, 
                                                  hallucination_rate, 
                                                  bad_output_format_rate)
from mlflow_experiment.inference.prompt_building.basic_user_prompt_builder import (
    BasicUserPromptBuilder,
)
from mlflow_experiment.inference.postprocessing.basic_postprocessing import (
    BasicPostprocessing,
)
from eda_utils.helpers import keep_beginning_and_ending
from mlflow_experiment.inference.model.basic_llama_cpp_model import BasicLlamaCppModel
import dotenv
import pandas as pd
from itertools import product

In [20]:
dotenv.load_dotenv("../.env",override=True)

True

In [21]:
data = pd.read_json("../data/subsample.jsonl", lines = True)
data.loc[:, "review"] = data.review.apply(lambda x: keep_beginning_and_ending(x, 30, 30))

In [22]:
keep_beginning_and_ending("This is an excellent, heartbreaking movie. It is by far the best I've seen that depicts the current reality in Latin America...kidnappings, corruption, ruthless and greedy police officials and heartless mayhem towards innocent victims. Denzel Washinton gives the most moving performance in his career, in my opinion. Dakota Fanning is an amazing young actress. The relationship between Washington and Fanning is wonderfully written and portrayed, I believed every minute. The cast is brilliant, Christopher Walken, Mickey Rourke are great as always. Walken lights up the screen for me like no other actor. I would have loved to see more of both of them. The authentic locations are remarkable. The camera work is interesting and different. There are many famous Latin actors in the cast, making it all the more interesting for people familiar with Latin American cinema. I highly recommend this movie.",.1,.3)

"This is an excellent, heartbreaking movie. It is by far the best I've seen both of them. The authentic locations are remarkable. The camera work is interesting and different. There are many famous Latin actors in the cast, making it all the more interesting for people familiar with Latin American cinema. I highly recommend this movie."

In [23]:
ev_pipeline = EvaluationPipeline(accuracy_score,
                                 precision,
                                 recall,
                                 false_negative_rate,
                                 false_positive_rate,
                                 median_response_token_count,
                                 median_query_token_count,
                                 median_processing_time,
                                 median_tokens_per_second,
                                 hallucination_rate,
                                 bad_output_format_rate,
                                 
                                )

In [37]:
MODEL_NAME = "Qwen2.5-1.5B-Instruct-Q5_K_M.gguf"
MODEL_PATH = f"../models/{MODEL_NAME}"
model = Llama(MODEL_PATH, verbose=False, n_ctx=32768, n_gpu_layers=-1, n_batch=2048)

prompt_builder = BasicUserPromptBuilder(
    """This is a movie review: {review}.
    ----
    Determine whether the overall sentiment of the review is positive or negative. You must look for evaluatory statements about the movie itself - e.g. "thrilling", "terrible acting", etc. These evaluatory statements are distinct from the movie genre! So be careful not to mistake descriptions about the genre. Take into account that different evaluatory statements might have different connation for different genres - e.g. a "terrifying movie" in the horror genre means a perfect movie, but in some other genre like commedy it might very well mean - rubbish movie. In the following you will recieve a few examples for each case.
    ----
    EXAMPLES:
    - NEGATIVE (negative): "Oh My Gosh!!!! This was the first movie Broken Lizard made as a group (although it just recently came to video), and I have never been more disappointed in my entire life!!! I tell you what, if I had seen this movie before I saw Super Troopers (which by the way, is a kick A$$ movie!!!), I never ever would have watched it!! I had read several reviews online, as well as on the cover of the DVD, that raved it as being,'Broken Lizard's funniest movie ever!' Now if they were referring to Super Troopers as being their funniest movie ever, I would agree nonstop, but not this one. Talk about dry. It took the movie a good 45 minutes to even get going, and by then, I was so out of the mood to watch it, that it wasn't even worth it. Maybe you gotta be high for it to really be funny? I dunno. I love these guys, I really do, but that movie is by far the worst one they've made. Club Dread was a pretty good movie, but this one, just wow. I'd highly recommend Super Troopers if you want a good laugh, but if you want more of a romance, drama, with a few funny spots, I'd say go with Puddle Cruiser. Just my opinion though, everyone is entitled to their own! :)"
    - POSITIVE (positive): "Now I myself had previously seen a few episodes of the Leauge Of Gentleman which I found hilarious. When I brought the film I was not sure if I knew enough about the series to get it, boy was I wrong. This is one of the best comedy films I have seen ever and the clever acting of the Leauge makes the film. It has a very good and funny plot as well as using only a few characters at any one time helps because it doesn't make it too confusing which would have wrecked the film. Even If you have never seen The Leauge Of Gentleman get this film it will make you laugh and this is a film that can be watched more then once and is an excellent film to watch with your mates. It truly deserves it review a definite 10 out of 10."
    - NEGATIVE (negative): "The animation was good, the imagery was good, although not totally original, however, the story was too long, way too confusing, and over the top dramatic. After about an hour I couldn't wait to get it over with. With so many characters that have nothing to contribute and plot elements that either come from nowhere or go nowhere this movie really wasn't one movie at all and would have been better of as a short series or possibly two movies. If you like this kind of typical story maybe you will like it, but frankly, I've been spoiled by much more creative stories that actually have some sort message to tell. Go rent a Miyazaki film and watch it twice, you'll get way more out of it."
    - POSITIVE (positive): ""This is an excellent, heartbreaking movie. It is by far the best I've seen that depicts the current reality in Latin America...kidnappings, corruption, ruthless and greedy police officials and heartless mayhem towards innocent victims. Denzel Washinton gives the most moving performance in his career, in my opinion. Dakota Fanning is an amazing young actress. The relationship between Washington and Fanning is wonderfully written and portrayed, I believed every minute. The cast is brilliant, Christopher Walken, Mickey Rourke are great as always. Walken lights up the screen for me like no other actor. I would have loved to see more of both of them. The authentic locations are remarkable. The camera work is interesting and different. There are many famous Latin actors in the cast, making it all the more interesting for people familiar with Latin American cinema. I highly recommend this movie.""
    ----
    If you are wrong 100000 little cute kittens will die a terrible death!!! YOU MUST BE CORRECT!
    ----
    Your output: your output should be either "positive" (if the overall sentiment is positive) or "negative" (if the overall sentiment is negative). Do not output nothing more. Output only "positive" or "negative"!
    """
)
llama_model = BasicLlamaCppModel(model, user_prompt_builder=prompt_builder, model_name = MODEL_NAME)

postprocessing_fn = BasicPostprocessing({"negative": 1, "positive": 0})

inf_pipeline = InferencePipeline(
    llama_model,
    system_prompt="You are a sentiment analysis system. Your goal is to categorize movie reviews into positive or negative. Follow the instructions given precisely!",
    postprocessing_fn=postprocessing_fn,
)



In [38]:
exp = MlflowExperiment(
    inference_pipeline=inf_pipeline,
    evaluation_pipeline=ev_pipeline,
    experiment_name="zero-shot-few-shot-prompting",
)

In [39]:
temperature = 0.2
top_k = 40
top_p = 0.95
inference_outputs, evaluation_results = exp.run(
    data[["review"]].to_dict(orient="records"),
    y_true=data.label,
    experiment_run_tags={"justification": "Provide description of the task without concrete examples. integer subsampling"},
    run_name=f"few-shot-int20-int20-kittens",
    temperature = temperature,
    top_k = top_k,
    top_p = top_p
)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1122/1122 [02:10<00:00,  8.57it/s]

🏃 View run few-shot-int20-int20-kittens at: http://localhost:5000/#/experiments/173405608153165389/runs/819fb2d4c3f9483a8dc57652a97ab0e5
🧪 View experiment at: http://localhost:5000/#/experiments/173405608153165389


In [36]:
evaluation_results

{'accuracy_score': 0.7745098039215687,
 'precision': np.float64(0.7015706806282722),
 'recall': np.float64(0.9554367201426025),
 'false_negative_rate': np.float64(0.0445632798573975),
 'false_positive_rate': np.float64(0.40641711229946526),
 'median_response_token_count': np.float64(1.0),
 'median_query_token_count': np.float64(1150.0),
 'median_processing_time': np.float64(0.06826984882354736),
 'median_tokens_per_second': np.float64(29.29550904035263),
 'hallucination_rate': np.float64(0.0),
 'bad_output_format_rate': np.float64(0.0)}

In [119]:
inference_outputs.loc[:, "true"] = data.label

In [120]:
inference_outputs

,output_string,label,is_hallucination,is_wrong_format,input_token_count,output_token_count,elapsed_time,true
0,positive,0,False,False,421,1,0.103696,0
1,negative,1,False,False,600,1,0.095301,1
2,negative,1,False,False,639,1,0.095017,1
3,negative,1,False,False,416,1,0.067424,0
4,positive,0,False,False,570,1,0.080919,0
...,...,...,...,...,...,...,...,...
1116,negative,1,False,False,394,1,0.058962,1
1117,positive,0,False,False,573,1,0.072158,0
1118,negative,1,False,False,800,1,0.083261,1
1119,positive,0,False,False,574,1,0.072770,0
